In [1]:
import sys
import os
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..'))
sys.path.append(parent_dir)

In [ ]:
from nfl_data_bowl_functions.data_imports import import_single_nfl_week_input, import_single_nfl_week_output, import_all_nfl_weeks_input, import_all_nfl_weeks_output
from function_library.py_data_visualizations.basic_data_descriptions import basic_dataframe_description
from nfl_data_bowl_functions.feature_engineering import calculate_receiver_separation_at_catch_point, calculate_defender_accelerometer_metric, calculate_receiver_ball_in_air_straightline_euclidean_efficiency

In [3]:
input_df = import_single_nfl_week_input(2023, 1, merge_supplementary=False)
output_df = import_single_nfl_week_output(2023, 1)

✓ 2023 Week 1 data: 285,714 play frames loaded
✓ 2023 Week 1 data: 32,088 play frames loaded


In [4]:
input_df.head()

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
0,2023090700,101,False,54527,1,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.09,0.39,322.40,238.24,21,63.259998,-0.22
1,2023090700,101,False,54527,2,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.04,0.61,200.89,236.05,21,63.259998,-0.22
2,2023090700,101,False,54527,3,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.93,0.12,0.73,147.55,240.60,21,63.259998,-0.22
3,2023090700,101,False,54527,4,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.35,36.92,0.23,0.81,131.40,244.25,21,63.259998,-0.22
4,2023090700,101,False,54527,5,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.37,36.90,0.35,0.82,123.26,244.25,21,63.259998,-0.22


In [5]:
basic_dataframe_description(input_df)


df Overview:
 Shape: (285714, 23)
 Columns: 23
 Memory: 141.96 MB
 Missing values: None
 Data types: {dtype('float64'): np.int64(8), dtype('int64'): np.int64(7), dtype('O'): np.int64(7), dtype('bool'): np.int64(1)}


,game_id,play_id,nfl_id,frame_id,absolute_yardline_number,player_weight,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
count,2.857140e+05,285714.000000,285714.000000,285714.000000,285714.000000,285714.000000,285714.000000,285714.000000,285714.000000,285714.000000,285714.000000,285714.000000,285714.000000,285714.000000,285714.000000
mean,2.023091e+09,2284.758118,49278.353829,16.055010,60.343049,211.784074,60.466355,26.751004,3.041966,2.126249,177.926268,181.198227,11.298669,60.851685,26.215487
std,8.575269e+01,1239.145629,5173.118005,10.689869,23.534325,21.965350,24.007917,10.035753,2.231126,1.430355,100.934248,98.972312,5.727945,25.759902,15.275341
min,2.023091e+09,55.000000,34452.000000,1.000000,11.000000,153.000000,1.210000,0.970000,0.000000,0.000000,0.000000,0.000000,5.000000,1.330000,-1.690000
25%,2.023091e+09,1232.000000,44995.000000,8.000000,40.000000,195.000000,41.630000,18.900000,1.120000,1.010000,89.530000,89.930000,8.000000,41.730000,12.640000
50%,2.023091e+09,2263.000000,48456.000000,15.000000,59.000000,208.000000,58.810000,26.710000,2.730000,1.920000,173.365000,182.230000,10.000000,59.060001,25.450001
75%,2.023091e+09,3302.000000,53609.000000,23.000000,79.000000,226.000000,78.520000,34.530000,4.650000,3.050000,269.580000,271.320000,13.000000,78.970001,39.250000
max,2.023091e+09,4699.000000,56663.000000,74.000000,109.000000,347.000000,119.860000,52.430000,12.530000,17.120000,359.990000,359.990000,94.000000,119.779999,57.330002


In [6]:
output_df.head()

,game_id,play_id,nfl_id,frame_id,x,y
0,2023090700,101,46137,1,56.22,17.28
1,2023090700,101,46137,2,56.63,16.88
2,2023090700,101,46137,3,57.06,16.46
3,2023090700,101,46137,4,57.48,16.02
4,2023090700,101,46137,5,57.91,15.56


In [7]:
basic_dataframe_description(output_df)


df Overview:
 Shape: (32088, 6)
 Columns: 6
 Memory: 1.47 MB
 Missing values: None
 Data types: {dtype('int64'): np.int64(4), dtype('float64'): np.int64(2)}


,game_id,play_id,nfl_id,frame_id,x,y
count,3.208800e+04,32088.000000,32088.000000,32088.000000,32088.000000,32088.000000
mean,2.023091e+09,2311.512746,49285.489529,8.405603,59.020316,26.530208
std,8.509051e+01,1216.723855,5089.708121,9.095645,25.550084,13.249616
min,2.023091e+09,55.000000,35459.000000,1.000000,0.420000,0.590000
25%,2.023091e+09,1337.000000,44962.000000,3.000000,40.567500,15.050000
50%,2.023091e+09,2283.000000,48241.000000,6.000000,58.430000,26.190000
75%,2.023091e+09,3285.000000,53819.000000,10.000000,77.000000,38.080000
max,2.023091e+09,4699.000000,56663.000000,94.000000,119.380000,52.910000


In [8]:
separation_metrics = calculate_receiver_separation_at_catch_point(input_df.sample(min(10000, len(input_df))), output_df)
separation_metrics.head(10)

,game_id,play_id,nearest_def_distance_catch_point,avg_def_distance_catch_point
0,2023091004,1839,4.425,4.425
1,2023091002,819,1.629,1.629
2,2023091011,209,5.824,7.969
3,2023091003,1929,2.807,5.560
4,2023091100,1186,0.969,6.989
5,2023091010,328,3.149,6.888
6,2023091100,629,13.055,13.055
7,2023091010,2236,10.150,12.450
8,2023091012,1713,2.468,7.403
9,2023091012,327,0.516,4.066


In [9]:
response_metrics = calculate_defender_accelerometer_metric(input_df, output_df)

In [10]:
response_metrics.head(10)

,game_id,play_id,defender_id,defender_accelerometer_metric
0,2023090700,101,46137,0.028282
1,2023090700,101,52546,0.029372
2,2023090700,194,44888,0.049488
3,2023090700,194,55910,0.020534
4,2023090700,194,53953,0.060450
5,2023090700,194,54653,0.022677
6,2023090700,219,44888,0.000577
7,2023090700,219,53953,0.086780
8,2023090700,361,46137,0.035242
9,2023090700,361,47882,0.029574


In [11]:
efficiency_metrics = calculate_receiver_ball_in_air_straightline_euclidean_efficiency(input_df, output_df)

In [12]:
efficiency_metrics.head(10)

,game_id,play_id,route_efficiency,total_distance,direct_distance
0,2023090700,101,1.130471,15.393935,17.402401
1,2023090700,194,0.766394,4.564542,3.498240
2,2023090700,219,1.011863,1.293282,1.308625
3,2023090700,361,1.285232,6.341358,8.150116
4,2023090700,436,1.053163,2.777491,2.925150
5,2023090700,461,0.987330,5.937323,5.862098
6,2023090700,530,0.644394,2.966988,1.911909
7,2023090700,621,1.029795,5.167620,5.321588
8,2023090700,713,0.755532,3.885375,2.935525
9,2023090700,736,0.822051,9.477241,7.790777
